In [1]:
%load_ext autoreload
%autoreload 2

import os
from dotenv import load_dotenv
from pathlib import Path
from research_tools.gpu import get_gpus_available

load_dotenv()


hf_access_token = os.getenv("HUGGINGFACE_API_KEY")

n_gpus = 1

gpus_available = get_gpus_available()
n_gpus = min(n_gpus, len(gpus_available))
gpus = gpus_available[:n_gpus]

assert n_gpus > 0, "No GPUs available"

os.environ["CUDA_VISIBLE_DEVICES"] = ",".join([str(i) for i in gpus])

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import List, Dict, Optional
import torch
from research_tools.utils import set_seed

set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
assert device == torch.device("cuda")

model_id = "HuggingFaceH4/zephyr-7b-beta"

# model = AutoModelForCausalLM.from_pretrained(
#     model_id, torch_dtype=torch.bfloat16, trust_remote_code=True, token=hf_access_token
# ).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = tokenizer.eos_token_id

In [3]:
from relearn.datasets.utils import (
    Datasets,
)
from relearn.datasets.corpus import process as process_corpus
from relearn.datasets.mcq import process as process_mcq
import pickle

data_dir = Path("../data")
cache_path = data_dir / "full.pickle"

assert cache_path.exists(), "Cache file does not exist"
with open(cache_path, "rb") as f:
    data = pickle.load(f)

In [4]:
from pathlib import Path
import pickle

folds_path = Path("../data/four.pickle")

assert folds_path.exists(), "Folds file does not exist"
with open(folds_path, "rb") as f:
    folds = pickle.load(f)

In [5]:
model_path = Path("../models/four/3")

In [ ]:
import wandb
from relearn.attacks import super_rtt
from relearn.attacks.folds import exponential_rtt

wandb.login()


config = {"model_id": model_id, "k_folds": 4}

run = wandb.init(
    project="relearn",
    config=config,
    tags=["exponential-rtt", "fold", "debug"],
    entity="12tqian",
)


model = exponential_rtt(
    model_path,
    tokenizer,
    folds,
    use_wandb=True,
)

run.finish()
# model = super_rtt(
#     model,
#     tokenizer,
#     data[Datasets.WMDP],
#     4,
#     use_wandb=True,
# )
# run.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: 12tqian. Use `wandb login --relogin` to force relogin


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

50it [00:18,  2.72it/s]
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 98/98 [00:09<00:00, 10.34it/s]
50it [00:17,  2.78it/s]
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 98/98 [00:09<00:00, 10.37it/s]
50it [00:17,  2.81it/s]
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 98/98 [00:09<00:00, 10.18it/s]
50it [00:18,  2.76it/s]
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 98/98 [00:09<00:00, 10.33it/s]
50it [00:17,  2.79it/s]
100%|███████████████████████████████████████████████████████████████████